# Data Exploration in Splice Machine
#### Let's start our Spark Session

In [ ]:
# Setup
from pyspark.sql import SparkSession
from splicemachine.spark import PySpliceContext
from splicemachine.mlflow_support.utilities import get_user

spark = SparkSession.builder.getOrCreate()
splice = PySpliceContext(spark)
schema = get_user()

import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")

## Ingesting Data
<blockquote><p class='quotation'><span style='font-size:15px'> There are a variety of ways to ingest data into splice machine— here we'll use an import from s3 directly into a splice machine table. From there we'll be able to investigate our features. <footer>Splice Machine</footer>
</blockquote>

In [ ]:
%%sql
drop table if exists cc_fraud_data; 
create table cc_fraud_data (
    time_offset integer,
    expected_weekly_trans_cnt double,
    expected_weekly_trans_amnt double,
    expected_daily_trans_cnt double,
    expected_daily_trans_amnt double,
    weekly_trans_cnt double,
    weekly_trans_amnt double,
    daily_trans_cnt double,
    daily_trans_amnt double,
    rolling_avg_weekly_trans_cnt double,
    rolling_avg_weekly_trans_amnt double,
    rolling_avg_daily_trans_cnt double,
    rolling_avg_daily_trans_amnt double,
    MACD_trans_amnt double,
    MACD_trans_cnt double,
    RSI_trans_amnt double,
    RSI_trans_cnt double,
    Aroon_trans_amnt double,
    Aroon_trans_cnt double,
    ADX_trans_amnt double,
    ADX_trans_cnt double,
    current_balance double,
    rolling_avg_balance double,
    MACD_balance double,
    Aroon_balance double,
    RSI_balance double,
    ADX_balance double,
    credit_score double,
    credit_limit double,
    amount decimal(10,2),
    class_result int
);

call SYSCS_UTIL.IMPORT_DATA (
     null,
     'cc_fraud_data',
     null,
     's3a://splice-demo/kaggle-fraud-data/creditcard.csv',
     ',',
     null,
     null,
     null,
     null,
     -1,
     's3a://splice-demo/kaggle-fraud-data/bad',
     null, 
     null);
     

## We can use `SQL` natively to investigate our data

In [ ]:
%%sql
select class_result, count(*) as NUM_OCurrences 
from cc_fraud_data 
group by class_result

In [ ]:
%%sql 
SELECT class_result, AVG(expected_weekly_trans_cnt) as avg_expected_weekly_trans_cnt, 
AVG(MACD_trans_amnt) as avg_MACD_trans_amnt, 
AVG(RSI_trans_amnt) as avg_RSI_trans_amnt
from cc_fraud_data
group by class_result

## Using the Native Spark Data Source to get our data in PySpark

In [ ]:
df = splice.df(f"SELECT * FROM {schema}.cc_fraud_data")

## Investigating the correlation among our features

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

pdf = df.filter(df.CLASS_RESULT == 0).limit(900).toPandas()\
        .append(df.filter(df.CLASS_RESULT == 1).limit(100).toPandas())
pdf = pdf.apply(pd.to_numeric)
corr = pdf.corr()

ticks = [i for i in range(len(corr.columns))]
mask = np.triu(np.ones_like(corr, dtype=np.bool))
f, ax = plt.subplots(figsize=(11, 9))
cmap = "coolwarm"

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.xticks(ticks, corr.columns)
plt.yticks(ticks, corr.columns)
plt.title('Credit Card Fraud Data Feature Correlation Heatmap')
plt.show()

## Investigating the distribution of our features

In [ ]:
features = list(pdf.columns)
features.remove("CLASS_RESULT")

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=6, figsize=(30,10))

for i in range(len(features)):
    row = int(i/6)
    col = i%6
    
    axes[row,col] = sns.distplot(pdf[features[i]], ax = axes[row,col])
    axes[row,col].set_xticks([])


## Investigating our features correlation to our target

In [ ]:
from beakerx.object import beakerx
beakerx.pandas_display_table()
most_correlated = corr.abs()['CLASS_RESULT'].sort_values(ascending=False).reset_index()
most_correlated = most_correlated.iloc[1:].rename({"index":"feature","CLASS_RESULT":"correlation_to_target"}, axis = 1)
print(most_correlated)

# Fantastic!
<blockquote> 
We've just demonstrated our platform's ability to execute simple feature investigation routines <br>
    Next Up: <a href='./7.4 Feature Engineering.ipynb'>Using MLManager to Engineer Features</a>
<footer>Splice Machine</footer>
</blockquote>